In [1]:
from train_sae import SAE
import torch
import transformers
import pandas as pd
from tqdm import tqdm

/home/ubuntu/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [52]:
df1 = pd.read_csv('/home/ubuntu/clip-text-directions/cc12m.tsv', sep='\t', header=None)
df2 = pd.read_parquet('/home/ubuntu/clip-text-directions/ori_prompts_df.parquet')
df2 = df2[['clean_prompts']]
df2.columns = ["text"]
df1=df1[[1]]
df1.columns = ["text"]
df = pd.concat([df1, df2], axis=0)
df=df.drop_duplicates()

In [66]:
df.to_parquet("full.parquet", index=False)

In [2]:
sae = SAE.resume("sae.pt")
sae = sae.to('cuda').to(torch.float16).requires_grad_(False)

In [3]:
df = pd.read_parquet("full.parquet")

In [5]:
# clip = transformers.CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14").to("cuda").to(torch.float16).requires_grad_(False)
# tokenizer = transformers.AutoTokenizer.from_pretrained("openai/clip-vit-large-patch14")

# def tokenize(x):
#     return tokenizer(x, padding="max_length", max_length=77, truncation=True, return_tensors="pt").input_ids


/home/ubuntu/miniconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
# texts = df.text.to_list()

In [5]:
embeds = torch.load("/home/ubuntu/clip-text-directions/all_embeds.pt")
embeds = torch.cat(embeds, dim=0).to(torch.float16).cuda()

In [9]:
most_active_features = []
batch_size = 4096
k = 10
torch_compile = False


# clip_forward = torch.compile(clip.forward)
if torch_compile:
    encode = torch.compile(sae.encode)
else:
    encode = sae.encode

with torch.no_grad():
    for i in tqdm(range(0, len(embeds), batch_size)):
        # input_ids = tokenize(texts[i:i+batch_size]).to(clip.device)
        # embs = clip_forward(input_ids).pooler_output
        embs = embeds[i:i+batch_size]
        # feat_indices = encode(embs).argmax(dim=-1).tolist()
        _, encoded_relu = encode(embs)
        feat_indices = encoded_relu.topk(k).indices.tolist()
        most_active_features.extend(feat_indices)

for i in range(k):
    most_active_feat = [f[i] for f in most_active_features]
    df[f'most_active_feature_{i}'] = most_active_feat
# df['most_active_feature'] = most_active_features

100%|██████████| 2796/2796 [00:25<00:00, 111.53it/s]


In [10]:
# get counts for each
counts = df.most_active_feature_0.value_counts()
counts.to_dict()

{24426: 1554121,
 17757: 290350,
 19077: 283369,
 15502: 264605,
 17401: 254258,
 8850: 253675,
 11738: 226099,
 6735: 220197,
 24152: 219200,
 7632: 198410,
 13964: 195576,
 3920: 185683,
 15701: 173309,
 22650: 170015,
 1133: 169166,
 22471: 167970,
 19597: 164126,
 1792: 153526,
 18061: 151218,
 16601: 143943,
 7521: 143822,
 9300: 141824,
 20545: 137366,
 13116: 134083,
 20198: 127676,
 8511: 126293,
 9293: 124303,
 15551: 124257,
 5884: 123720,
 19406: 121075,
 16907: 120720,
 3935: 119158,
 11661: 115452,
 23259: 114498,
 8111: 112639,
 24208: 112292,
 18828: 103027,
 2275: 102532,
 844: 99764,
 2397: 98659,
 12431: 96032,
 12442: 93119,
 14651: 91830,
 15095: 85303,
 13007: 84572,
 20273: 83040,
 11562: 82707,
 19149: 80123,
 11613: 79444,
 24475: 78040,
 10116: 75911,
 14332: 73673,
 13259: 72831,
 7451: 71512,
 11625: 68126,
 1179: 66822,
 5828: 66774,
 10167: 65212,
 10968: 61996,
 2358: 59945,
 1392: 58247,
 2799: 57990,
 16529: 56295,
 23124: 55831,
 22673: 55080,
 12429: 5

In [29]:
df[df.most_active_feature_0 == 3920].sample(10).text.to_list()

['fullbody urban photograph from the mid90s with a lot of neon colors of 35 years old, laughing Til Schweiger leaning on the front engine hood of a yellow sports car in the summer sun with blue sky in the background. His is arm wrapped around his 35 years old buddy with semilong brown hair who is wearing an open plaid flannel shirt. Neon colors, colorful, shot on Agfa Vista 200, 4k ',
 'close up photograph, torso, extremely attractive woman, at the beach, clear plastic outfit, smiling, trashy, confident, disheveled, sassy, minx, cinematic film still, kodak portra ',
 'Red panda cub sits in a tree in the zoo Zlin, Czech Republic, November 3, 2017. (CTK Photo/ <PERSON>)',
 'Aerial footage of traffic travelling across the Queensferry Crossing bridge at South Queensferry on a sunny day in West Lothian, Scotland',
 'Cartoon funny baby taking a bath with foam, vector illustration. Cartoon funny baby taking a bath with foam white background, vector illustration royalty free illustration',
 'S

In [31]:
df.to_parquet("full.parquet",index=False)